In [5]:
!pip install -qU --upgrade pip 
!pip install -qU "weaviate-client==3.26.0"
!pip install -qU "openai==0.27.7"


done


In [6]:
import weaviate
import json
import os
import sys
import logging
import openai

openai_api_key=os.environ.get('OPENAI_API_KEY')
file_path = '/home/jovyan/work/wiki.txt'
limit = 3750

# ログレベルの設定
logging.basicConfig(stream=sys.stdout, level=logging.INFO, force=True)

client = weaviate.Client(
    # url = "https://your-sandbox.weaviate.network",  # Replace with your endpoint
    url = "http://weaviate:8080/",  # Replace with your endpoint
    # auth_client_secret=weaviate.AuthApiKey(api_key= weaviate_api_key),  # Replace w/ your Weaviate instance API key
    additional_headers = {
        "X-OpenAI-Api-Key": openai_api_key
    }
)

client.is_ready()


True

In [7]:
client.schema.delete_class('Question')
client.batch.empty_objects()

class_obj = {
    "class": "Description",
    "vectorizer": "text2vec-openai",  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
    "moduleConfig": {
        "text2vec-openai": {},
        "generative-openai": {}  # Ensure the `generative-openai` module is used for generative queries
    }
}

client.schema.create_class(class_obj)


In [8]:
# 配列を初期化
texts = []

# ファイルを開く
with open(file_path, 'r') as file:
    for line in file:
        if len(line) > 3:
            texts.append(line.strip())

In [9]:
import requests
import json
# resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
# data = json.loads(resp.text)  # Load data

client.batch.configure(batch_size=100)  # Configure batch
with client.batch as batch:  # Initialize a batch process
    for i, d in enumerate(texts):
        print(f"importing description: {i+1}")
        properties = {
            "description": d
            # "answer": d["Answer"],
            # "question": d["Question"],
            # "category": d["Category"],
        }
        batch.add_data_object(
            data_object=properties,
            class_name="Description"
        )

importing description: 1
importing description: 2
importing description: 3
importing description: 4
importing description: 5
importing description: 6
importing description: 7
importing description: 8
importing description: 9
importing description: 10
importing description: 11
importing description: 12
importing description: 13
importing description: 14
importing description: 15
importing description: 16
importing description: 17
importing description: 18
importing description: 19
importing description: 20
importing description: 21
importing description: 22
importing description: 23
importing description: 24
importing description: 25
importing description: 26
importing description: 27
importing description: 28
importing description: 29
importing description: 30
importing description: 31
importing description: 32
importing description: 33
importing description: 34
importing description: 35
importing description: 36
importing description: 37
importing description: 38
importing description

In [14]:
def retrieve(query):

    response = (
        client.query
        .get("Description", ["description"])
        .with_near_text({
            "concepts": [f"{query}"]
        })
        .with_limit(3)
        .do()
    )

    x=json.dumps(response, indent=4).encode().decode('unicode_escape')
    # print(x)
    
    # get relevant contexts
    # contexts = [response['data']['Get']['Description']]
    contexts = [
        x['description'] for x in response['data']['Get']['Description']
    ]

    # build our prompt with the retrieved contexts included
    prompt_start = (
        "Answer the question based on the context below.\n\n"+
        "Context:\n"
    )
    prompt_end = (
        f"\n\nQuestion: {query}\nAnswer:"
    )
    prompt = []
    # append contexts until hitting limit
    for i in range(1, len(contexts)):
        if len("\n\n---\n\n".join(contexts[:i])) >= limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts) +
                prompt_end
            )
    print(prompt)
    return prompt

def complete(prompt):
    # query text-davinci-003
    res = openai.Completion.create(
        engine='text-davinci-003',
        prompt=prompt,
        temperature=0,
        max_tokens=400,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    results = res['choices'][0]['text'].strip()
    print(results)
    return results

prompt = retrieve('please write your question')
complete(prompt)

Answer the question based on the context below.

Context:
青鬼は物語の主人公である。入間の村で雑貨屋さんを経営している。その昔は川で渡し船を営んでいた。青鬼はどんな洋服でもイメージするだけでつくることができる特殊能力を持っている。地味だがこの能力のお陰でピンチを切り抜けたことがある。主人公キャラと自分では思っていて、事件はいつも周りに巻き込まれて始まる事が多い。

---

青鬼は青鬼とおにぎり工場に行ったことがある。おにぎり工場にいくと、試食室があった、部屋に入ると服を脱いだり、体に塩を塗ったりと色々な指示をされる。おにぎり工場は実は、鬼を食べてしまう、鬼切りの工場だった。青鬼と赤鬼はなんとか逃げることができた。

---

カッパは川に住んでいる。青鬼の友達で、ときどき青鬼の船に載せてくれと頼んでくる。お金はちゃんと払ってくれるがいつもカッパの粘液でぬるぬるである。「ぬるぬる小判」の回では青鬼の盗まれた舟を一緒に探してくれ、殿様の小判を拾ってくれるが、カッパの粘液でぬるぬるだった。

Question: 青鬼は昔渡船をやっていました。今はどんな仕事をやっているでしょうか？
Answer:
青鬼は入間の村で雑貨屋さんを経営している。


'青鬼は入間の村で雑貨屋さんを経営している。'